In [ ]:
import pandas as pd
import numpy as np
import json
import time
pd.set_option('display.max_colwidth', 255)
from openai import OpenAI,DefaultHttpxClient

client = OpenAI(
    api_key="sk-yourkey",
    http_client=DefaultHttpxClient(
        verify=False
    ),
)

In [ ]:
## load clustered GBB or source fields dataframe
df = pd.read_parquet('cluster_results_tobenamed.parquet')
df['clus'] = df.clus.str[1:].astype(int)
df = df[df.clus!=-1]
df.head()

,cpc,mean,max,sum,count,cpc1d,cpc4d,title,x,y,clus,clus2
0,A01B1/00,0.864104,0.877908,1.728208,2,A,A01B,Hand tools,3.285313,-2.539227,10,c1
8,A01B39/18,0.837579,0.837579,0.837579,1,A,A01B,"Other machines specially adapted for working soil on which crops are growing-for special purposes ; , e.g. for special culture-for weeding",3.493809,-2.734650,10,c1
9,A01B61/00,0.840016,0.840016,0.840016,1,A,A01B,"Devices for, or parts of, agricultural machines or implements for preventing overstrain",3.526913,-2.810171,10,c1
11,A01B63/1013,0.820958,0.820958,0.820958,1,A,A01B,Lifting or adjusting devices or arrangements for agricultural machines or implements -for implements mounted on tractors-operated by hydraulic or pneumatic means-the hydraulic or pneumatic means structurally belonging to the tractor-and being located ...,2.154277,-5.402795,10,c65
12,A01B69/008,0.728450,0.728450,0.728450,1,A,A01B,"Steering of agricultural machines or implements; Guiding agricultural machines or implements on a desired track-Steering or guiding of agricultural vehicles, e.g. steering of the tractor to keep the plough in the furrow-automatic",3.437483,-2.810456,10,c1


In [3]:
df = (
    df.dropna()
    .rename(columns={"title": "tech", "mean": "weight"})
    .sort_values(["clus", "weight"], ascending=False)
    .reset_index(drop=True)
    .copy()
)
df["weight"] = df["weight"].round(3)
df["tech"] = df["tech"].str.replace('"', "").str.replace("'", "")
df.head()

,cpc,weight,max,sum,count,cpc1d,cpc4d,tech,x,y,clus,clus2
0,H01H3/28,0.976,0.975758,0.975758,1,H,H01H,Mechanisms for operating contacts -Power arrangements internal to the switch for operating the driving mechanism-using electromagnet,-2.765656,6.364182,81,c72
1,G01R23/16,0.901,0.901429,0.901429,1,G,G01R,Arrangements for measuring frequencies; Arrangements for analysing frequency spectra-Spectrum analysis; Fourier analysis,-1.971989,7.487788,81,c72
2,G01R29/08,0.895,0.895254,0.895254,1,G,G01R,Arrangements for measuring or indicating electric quantities not covered by groups G01R19/00 - G01R27/00-Measuring electromagnetic field characteristics,-1.992803,7.491471,81,c72
3,G01R19/04,0.894,0.894090,0.894090,1,G,G01R,Arrangements for measuring currents or voltages or for indicating presence or sign thereof -Measuring peak values ; or amplitude or envelope; of ac or of pulses,-1.960905,7.457569,81,c72
4,G01R31/42,0.884,0.883618,0.883618,1,G,G01R,Arrangements for testing electric properties; Arrangements for locating electric faults; Arrangements for electrical testing characterised by what is being tested not provided for elsewhere -Testing power supplies -AC power supplies,-2.108531,7.352282,81,c72


In [4]:
prompt_template = '''
Please name the cluster of technologies from the descriptions, the data is in following format:
[{{"tech":"description of technology", "weight":"weight of technology"}},
{{"tech":"description of another technology", "weight":'weight of another technology"}}]

Weight ranges from -1 to 1, higher weight value means you should put more emphasis on the corresponding description of technology.

You should only answer in the following json format:
{{"name":"name you assigned to the cluster, less than 5 words","desc":"summarized description of the cluster, less than 20 words"}}

Data:
{0}

Your answer in plain text without code block syntax around it:
'''

In [5]:
df_clus = (
    df[df.clus!='c-1'].groupby("clus")
    .head(30)
    .groupby("clus")[["tech", "weight"]]
    .apply(lambda x: pd.Series({'prompt':prompt_template.format(json.dumps(x.to_dict(orient="records")))}))
    .reset_index()
)
df_clus['gpt']=''
df_clus.tail()

,clus,prompt,gpt
77,77,"\nPlease name the cluster of technologies from the descriptions, the data is in following format:\n[{""tech"":""description of technology"", ""weight"":""weight of technology""},\n{""tech"":""description of another technology"", ""weight"":'weight of another techno...",
78,78,"\nPlease name the cluster of technologies from the descriptions, the data is in following format:\n[{""tech"":""description of technology"", ""weight"":""weight of technology""},\n{""tech"":""description of another technology"", ""weight"":'weight of another techno...",
79,79,"\nPlease name the cluster of technologies from the descriptions, the data is in following format:\n[{""tech"":""description of technology"", ""weight"":""weight of technology""},\n{""tech"":""description of another technology"", ""weight"":'weight of another techno...",
80,80,"\nPlease name the cluster of technologies from the descriptions, the data is in following format:\n[{""tech"":""description of technology"", ""weight"":""weight of technology""},\n{""tech"":""description of another technology"", ""weight"":'weight of another techno...",
81,81,"\nPlease name the cluster of technologies from the descriptions, the data is in following format:\n[{""tech"":""description of technology"", ""weight"":""weight of technology""},\n{""tech"":""description of another technology"", ""weight"":'weight of another techno...",


In [8]:
iternum = 0
while (df_clus.gpt == "").sum() > 0 and iternum < 5:
    for i in df_clus[df_clus.gpt == ""]["clus"]:
        try:
            res = (
                client.chat.completions.create(
                    model="gpt-4o",
                    messages=[
                        {
                            "role": "user",
                            "content": df_clus["prompt"][i],
                        }
                    ]
                )
                .choices[0]
                .message.content
            )
        except:
            res = ""
        df_clus.loc[i, "gpt"] = res
        if res == "":
            time.sleep(2)
    iternum = iternum + 1
    print(iternum)
    time.sleep(1 * iternum)

1


In [9]:
df_clus[df_clus.gpt=='']

,clus,prompt,gpt


In [ ]:
df_clus[['name','desc']] = df_clus.gpt.apply(lambda x: pd.Series(json.loads(x)))

In [ ]:
clusname = df_clus[['clus','name','desc']].drop_duplicates()

In [6]:
clusname.to_parquet('data/clusname3.parquet')